# Overview
Use this notebook to convert videos into jpgs and annotate . Opencv comes pre-compiled through homebrew which is way easier than compiling by hand. Install homebrew and then run 'brew install opencv@2'

## Split videos to file. 
Convert each video file into images and name them as sequential numbers. This will make it easy to annotate them which you'll see how to do once we have the image files. 

In [3]:
import cv2
import os
from PIL import Image

def video_to_frames(video, path_output_dir):

    # extract frames from a video and save to directory as 'x.png' where 
    # x is the frame index
    vidcap = cv2.VideoCapture(video)
    count = 0
    while vidcap.isOpened():
        success, image = vidcap.read()
        if success:
            # Add padding on these image files so that 10 does't convert to 1 and erase images. 
            cv2.imwrite(os.path.join(path_output_dir, '{0:05d}.jpg'.format(count)), image)
            count += 1
        else:
            break
    cv2.destroyAllWindows()
    vidcap.release()

In [8]:
path_to_video_dir = './movies/sideview/'
output_dir_base = './data/sideview/img/'

# Iterate through every file in the video directory
for file in os.listdir(path_to_video_dir):
    # Append the filename to the path and create the directory structure if doesn't exist
    output_dir = os.path.join(output_dir_base, file.split('.')[0])
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Split every video into images and dump into created directories. 
    video_to_frames(os.path.join(path_to_video_dir, file), output_dir)

## Annotate Files
At this point, you should have a bunch of image files dumped into the directories we just created. Now we need to annotate them. At the heart of the problem, we're just trying to determine if the glass is full or not. This means we only have two classifications: Full or not full. Because we video'd the cuups while they were being filled, there's a point in the video where the glass changes state from not full to full. We will locate that point manually and note the image number that this change happens. Once we have that number, we'll run the code below to prepend our classification to the file name and use it to annotate these files when we pipe it into the model for training. 

In [53]:
def add_prefix_to_files(dir_path, empty_cutoff, output_dir):
    # Filter out the hidden files and directories like .DStore 
    images = filter( lambda f: not f.startswith('.'), os.listdir(dir_path))
    
    # create_naming_schema
    for i, img_name in enumerate(images):
        filepath = os.path.join(dir_path, img_name)
        # We want the image_src directory to make the name unique since each directory would 
        # Have empty_00000.jpg for example
        img_src = dir_path.split('/')[-1]
        prefix = 'empty' if i <= empty_cutoff else 'full'
        new_filename = '{}_{}_{}'.format(prefix, img_src, img_name)
        new_filepath = os.path.join(output_dir, new_filename)
        os.rename(filepath, new_filepath)

### Prefix annotation to file names
This step requires some manual work. Locate the point where the cup becomes full and input it into the add_prefix_to_files() function above. Since each directory has a different cutoff, it's easier to switch the directory path and index then re-run the code below every time for each directory. 

In [61]:
# dir_path = './data/sideview/img/sideview_6'
# output_dir = './data/sideview/annotated'
# empty_cutoff = 384

# add_prefix_to_files(dir_path, empty_cutoff, output_dir)

## Generate .rec files to use for mxnet model
Below we will split up the files annotated previously into a training and validation set. 

In [1]:
class_map = {
    'empty': [],
    'full': []
}

class_to_idx = {
    'empty': 0,
    'full': 1
}

Take all the images we've annotated, pull their annotations from the filename, and sort them into their corresponding category

In [4]:
data_dir = './data/sideview/annotated'
# Get files from directory and ignore hidden files like .DS files
names = filter( lambda f: not f.startswith('.'), os.listdir(data_dir))
counter = 0
for fn in names:
    arr = fn.split('_')
    category = arr[0]
    class_map[category].append({
        'idx': counter,
        'label': class_to_idx[category],
        'filename': fn,
    })
    counter += 1
    
for k in class_map:
    print(k, len(class_map[k]))

empty 980
full 1167


Shuffle the images so that our batches when training the model have an even distribution of different types of training samples

In [6]:
import random
for k in class_map:
    random.shuffle(class_map[k])

In [90]:
def write_lst(image_arr, base_dir, file_path):
    with open(file_path, 'w') as f:
        count = 0
        for img in image_arr:
            label = img['label']
            img_path = os.path.join(base_dir, img['filename'])
            new_line = '\t'.join([str(count), str(label), str(img_path)])
            new_line += '\n'
            f.write(new_line)
            count += 1

Flatten the items from each class into one big dataset adn split it up 80/20 for training and validation sets. 

In [7]:
#Get full dataset
flatten = lambda l: [item for sublist in l for item in sublist]
full_dataset = flatten([class_map[k] for k in class_map])
random.shuffle(full_dataset)

train = (0, int(len(full_dataset) * 0.8))
validation = (int(len(full_dataset) * 0.8), int(len(full_dataset) * 1))

In [8]:
train_set = full_dataset[train[0]: train[1]]
validation_set = full_dataset[validation[0]: validation[1]]
print(len(train_set))

1717


## Generate lst files
The lst file will be used by the im2rec.py script mxnet provides in order to create the rec files that mxnet will use as input into the model 

In [93]:
if not os.path.exists('./data/train'):
    os.makedirs('./data/train')
    
write_lst(train_set, data_dir, './data/train/train.lst')

In [94]:
if not os.path.exists('./data/validation'):
    os.makedirs('./data/validation')
    
write_lst(validation_set, './data/img', './data/validation/validation.lst')

## Run generate_rec.sh
this script will generate rec file based on lst for different dataset

* python3 im2rec.py ./data/train . --center-crop True --resize 512 --pack-label True
* python3 im2rec.py ./data/validation . --center-crop True --resize 512 --pack-label True